In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
# First, we get the 800 timepoints we are considering as a list, to use for indices given by cluster-based permutation later
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')
pp = 0 #only need one participant

# we can use aligned data, any movement condition (e.g. leftmoves)

id_directory = os.path.join(data_directory, 'p%03d/' % pp)
pp_directory = os.path.join(id_directory, erps)
fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_%s-ave.fif' % (pp, 'leftmoves', 'aligned', 'blockall'))
evoked = mne.read_evokeds(fname)
df = evoked[0].to_data_frame()
time = df['time'].tolist()

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)


In [3]:
# Load data for each condition
# Create separate lists for C3 and C4, along with right and left movements, so that we may calculate for LRPs
# transform data into array accepted for cluster-based permutation test in mne
# output: (n_participants, n_timepts) for each condition

root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

# #specify channels we need - LRP
channels = ['C3', 'C4']

#ALIGNED
#read in evoked object
movenames = ['rightmoves', 'leftmoves']

for channel in range(0, len(channels)):
    for move in range(0, len(movenames)):
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], movenames[move], 'aligned', 'all'))
            evoked = mne.read_evokeds(fname)
            evoked = evoked[0]
            evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
            evoked = evoked[0] #get first list then index timepts below
            evoked = evoked[100:401] #get timepts we are interested in
            evoked_list.append(evoked)
        if move == 0 and channel == 0:
            aln_right_C3 = evoked_list
        elif move == 0 and channel == 1:
            aln_right_C4 = evoked_list
        elif move == 1 and channel == 0:
            aln_left_C3 = evoked_list
        elif move == 1 and channel == 1:
            aln_left_C4 = evoked_list
            
#ALIGNED FOR ROTATION
movenames = ['rightmoves', 'leftmoves']

for channel in range(0, len(channels)):
    for move in range(0, len(movenames)):
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], movenames[move], 'aln_rot', 'all'))
            if os.path.isfile(fname) == False:
                continue
            evoked = mne.read_evokeds(fname)
            evoked = evoked[0]
            evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
            evoked = evoked[0] #get first list then index timepts below
            evoked = evoked[100:401] #get timepts we are interested in
            evoked_list.append(evoked)
        if move == 0 and channel == 0:
            alnrot_right_C3 = evoked_list
        elif move == 0 and channel == 1:
            alnrot_right_C4 = evoked_list
        elif move == 1 and channel == 0:
            alnrot_left_C3 = evoked_list
        elif move == 1 and channel == 1:
            alnrot_left_C4 = evoked_list
            
#ALIGNED FOR MIRROR
movenames = ['rightmoves', 'leftmoves']

for channel in range(0, len(channels)):
    for move in range(0, len(movenames)):
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], movenames[move], 'aln_mir', 'all'))
            if os.path.isfile(fname) == False:
                continue
            evoked = mne.read_evokeds(fname)
            evoked = evoked[0]
            evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
            evoked = evoked[0] #get first list then index timepts below
            evoked = evoked[100:401] #get timepts we are interested in
            evoked_list.append(evoked)
        if move == 0 and channel == 0:
            alnmir_right_C3 = evoked_list
        elif move == 0 and channel == 1:
            alnmir_right_C4 = evoked_list
        elif move == 1 and channel == 0:
            alnmir_left_C3 = evoked_list
        elif move == 1 and channel == 1:
            alnmir_left_C4 = evoked_list
            
#PERTURBED CONDITIONS - only half of N in rot and mir due to considering only left and right movements
conditionnames = ['early', 'late']

#ROTATED
for condition in range(0, len(conditionnames)):
    for channel in range(0, len(channels)):
        for move in range(0, len(movenames)):
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], movenames[move], 'rot', condition))
                if os.path.isfile(fname) == False:
                    continue
                evoked = mne.read_evokeds(fname)
                evoked = evoked[0]
                evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
                evoked = evoked[0] #get first list then index timepts below
                evoked = evoked[100:401] #get timepts we are interested in
                evoked_list.append(evoked)
            if move == 0 and channel == 0 and condition == 0:
                rot_early_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 0:
                rot_early_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 0:
                rot_early_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 0:
                rot_early_left_C4 = evoked_list
            elif move == 0 and channel == 0 and condition == 1:
                rot_late_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 1:
                rot_late_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 1:
                rot_late_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 1:
                rot_late_left_C4 = evoked_list

#MIRROR
for condition in range(0, len(conditionnames)):
    for channel in range(0, len(channels)):
        for move in range(0, len(movenames)):
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], movenames[move], 'mir', condition))
                if os.path.isfile(fname) == False:
                    continue
                evoked = mne.read_evokeds(fname)
                evoked = evoked[0]
                evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
                evoked = evoked[0] #get first list then index timepts below
                evoked = evoked[100:401] #get timepts we are interested in
                evoked_list.append(evoked)
            if move == 0 and channel == 0 and condition == 0:
                mir_early_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 0:
                mir_early_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 0:
                mir_early_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 0:
                mir_early_left_C4 = evoked_list
            elif move == 0 and channel == 0 and condition == 1:
                mir_late_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 1:
                mir_late_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 1:
                mir_late_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 1:
                mir_late_left_C4 = evoked_list

#RANDOM
for condition in range(0, len(conditionnames)):
    for channel in range(0, len(channels)):
        for move in range(0, len(movenames)):
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_EarlyLate_%s_%s_block%s-ave.fif' % (participants[pp], movenames[move], 'rdm', condition))
                if os.path.isfile(fname) == False:
                    continue
                evoked = mne.read_evokeds(fname)
                evoked = evoked[0]
                evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
                evoked = evoked[0] #get first list then index timepts below
                evoked = evoked[100:401] #get timepts we are interested in
                evoked_list.append(evoked)
            if move == 0 and channel == 0 and condition == 0:
                rdm_early_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 0:
                rdm_early_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 0:
                rdm_early_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 0:
                rdm_early_left_C4 = evoked_list
            elif move == 0 and channel == 0 and condition == 1:
                rdm_late_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 1:
                rdm_late_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 1:
                rdm_late_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 1:
                rdm_late_left_C4 = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the dat

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p019\lrp\p019_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\lrp\p020_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\lrp\p021_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the dat

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p006\lrp\p006_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\lrp\p007_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\lrp\p008_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data o

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p025\lrp\p025_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\lrp\p026_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\lrp\p027_EarlyLate_leftmoves_aligned_blockall-ave.fif ...
    Found the data o

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p012\lrp\p012_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p013\lrp\p013_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p014\lrp\p014_EarlyLate_rightmoves_aligned_blockall-ave.fif ...
    Found the dat

...

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - a

        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\lrp\p020_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\lrp\p021_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is basel

Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\lrp\p007_EarlyLate_rightmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\lrp\p008_EarlyLate_rightmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\lrp\p009_EarlyLate_r

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\lrp\p027_EarlyLate_rightmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p028\lrp\p028_EarlyLate_rightmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 -

        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p014\lrp\p014_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p015\lrp\p015_EarlyLate_leftmoves_rdm_block1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is basel

In [4]:
#Next, we calculate LRPs: (right C3 - right C4) - (left C3 - left C4)

perturbations = ['aln', 'alnrot', 'alnmir', 'rot', 'rdm', 'mir']
conditionnames = ['early', 'late']

for p in range(0, len(perturbations)):
    if p == 0:
        rightdiff = np.subtract(aln_right_C3, aln_right_C4)
        leftdiff = np.subtract(aln_left_C3, aln_left_C4)
        aln_lrp = np.subtract(rightdiff, leftdiff)
    elif p == 1:
        rightdiff = np.subtract(alnrot_right_C3, alnrot_right_C4)
        leftdiff = np.subtract(alnrot_left_C3, alnrot_left_C4)
        alnrot_lrp = np.subtract(rightdiff, leftdiff)
    elif p == 2:
        rightdiff = np.subtract(alnmir_right_C3, alnmir_right_C4)
        leftdiff = np.subtract(alnmir_left_C3, alnmir_left_C4)
        alnmir_lrp = np.subtract(rightdiff, leftdiff)
    elif p == 3:
        for condition in range(0, len(conditionnames)):
            if condition == 0:
                rightdiff = np.subtract(rot_early_right_C3, rot_early_right_C4)
                leftdiff = np.subtract(rot_early_left_C3, rot_early_left_C4)
                early_rot_lrp = np.subtract(rightdiff, leftdiff)
            elif condition == 1:
                rightdiff = np.subtract(rot_late_right_C3, rot_late_right_C4)
                leftdiff = np.subtract(rot_late_left_C3, rot_late_left_C4)
                late_rot_lrp = np.subtract(rightdiff, leftdiff)
    elif p == 4:
        for condition in range(0, len(conditionnames)):
            if condition == 0:
                rightdiff = np.subtract(rdm_early_right_C3, rdm_early_right_C4)
                leftdiff = np.subtract(rdm_early_left_C3, rdm_early_left_C4)
                early_rdm_lrp = np.subtract(rightdiff, leftdiff)
            elif condition == 1:
                rightdiff = np.subtract(rdm_late_right_C3, rdm_late_right_C4)
                leftdiff = np.subtract(rdm_late_left_C3, rdm_late_left_C4)
                late_rdm_lrp = np.subtract(rightdiff, leftdiff)
    elif p == 5:
        for condition in range(0, len(conditionnames)):
            if condition == 0:
                rightdiff = np.subtract(mir_early_right_C3, mir_early_right_C4)
                leftdiff = np.subtract(mir_early_left_C3, mir_early_left_C4)
                early_mir_lrp = np.subtract(rightdiff, leftdiff)
            elif condition == 1:
                rightdiff = np.subtract(mir_late_right_C3, mir_late_right_C4)
                leftdiff = np.subtract(mir_late_left_C3, mir_late_left_C4)
                late_mir_lrp = np.subtract(rightdiff, leftdiff)

In [ ]:
# Then we can run cluster based permutation tests, comparing our conditions
# Note, rot and mir conditions only have 16 participants, while aligned and random have 32
# Hence we grab corresponding aln participants for rot and mir above

# Comparing two ERP signals is just the same as taking their difference (ERP1 minus ERP2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
    np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0